# Train you first forecasting model

So you picked the forecasting project. Good for you! Forecasting is an important skill, and it's helpful to learn about the techchnologies out there that can make your life easier. Or worse, depending on how you look at it. ;)

### 1. Download the Kaggle forecasting data
Use the notebook example for this.

### 2. Access your data

In [97]:
import pandas as pd
import json
import sagemaker
import boto3
import numpy as np

HEAD is now at 43a9be7 Merge branch 'master' of https://github.com/0Architectus0/aws_arch4ML


In [26]:
!pwd

/home/ec2-user/SageMaker/aws_arch4ML/script/ETL/ex


In [28]:
df = pd.read_csv('/home/ec2-user/SageMaker/aws_arch4ML/data/data/train.csv')

In [29]:
df.shape

(5113, 99)

In [30]:
df.head()

,Date,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,...,VINI,WASH,WATO,WAUR,WEAT,WEST,WILB,WIST,WOOD,WYNO
0,19940101,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,...,10771800,12116400,11308800,12361800,11331600,10644300,11715600,11241000,10490100,10545300
1,19940102,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,...,4314300,10733400,9154800,12041400,9168300,4082700,9228000,5829900,7412100,3345300
2,19940103,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,...,2976900,11775000,10700400,12687300,11324400,2746500,3686700,4488900,9712200,4442100
3,19940104,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,...,3476400,12159600,11907000,12953100,11903700,2741400,4905000,4089300,11401500,4365000
4,19940105,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,...,6393300,11419500,7334400,10178700,7471500,8235300,11159100,10651500,10006200,8568300


In [48]:
df['Date'].dtypes

dtype('int64')

Each row here is a new point in time, and each column is an energy station. That means that each COLUMN is a unique time series data set. We are going to train our first model on a single column. Then, you can extend it by adding more columns.

In [31]:
target = df['ACME']

In [32]:
target.shape

(5113,)

Great! We have 5113 obervations. That's well over the 300-limit on DeepAR.

In [82]:
station_info = pd.read_csv('/home/ec2-user/SageMaker/aws_arch4ML/data/data/station_info.csv')

train_info = pd.read_csv('/home/ec2-user/SageMaker/aws_arch4ML/data/data/train.csv')

test_info = pd.read_csv('/home/ec2-user/SageMaker/aws_arch4ML/data/data/test.csv')

In [126]:
dfSampSub.shape

(98, 4)

In [61]:
import netCDF4 as nc

In [124]:
def load_GEFS_file(directory,data_type,file_sub_str):
    print('loading',data_type)
    path = os.path.join(directory,data_type+file_sub_str)
    X = list(nc.Dataset(path,'r+').variables.values())[-1][:,:,:,3:7,3:13] # get rid of some GEFS points
    #X = list(nc.Dataset(path,'r+').variables.values())[-1][:]
    #X = X.reshape(X.shape[0],55,4,10) 								 # Reshape to merge sub_models and time_forcasts
    X = np.mean(X,axis=1) 											 # Average models, but not hours
    X = X.reshape(X.shape[0],np.prod(X.shape[1:])) 					 # Reshape into (n_examples,n_features)
    return X

In [123]:
def load_GEFS_data(directory,files_to_use,file_sub_str):
    for i,f in enumerate(files_to_use):
        if i == 0:
            X = load_GEFS_file(directory,files_to_use[i],file_sub_str)
        else:
            X_new = load_GEFS_file(directory,files_to_use[i],file_sub_str)
            X = np.hstack((X,X_new))
    return X

In [128]:
X=load_GEFS_data("/home/ec2-user/SageMaker/aws_arch4ML/data/data/gefs-train",["apcp_sfc","dlwrf_sfc","dswrf_sfc","pres_msl","pwat_eatm","spfh_2m","tcdc_eatm",
                                   "tcolc_eatm","tmax_2m","tmin_2m","tmp_2m","tmp_sfc","ulwrf_sfc","ulwrf_tatm"],
               "_latlon_subset_20080101_20121130.nc")

loading apcp_sfc
loading dlwrf_sfc
loading dswrf_sfc
loading pres_msl
loading pwat_eatm
loading spfh_2m
loading tcdc_eatm
loading tcolc_eatm
loading tmax_2m
loading tmin_2m
loading tmp_2m
loading tmp_sfc
loading ulwrf_sfc
loading ulwrf_tatm


In [134]:
X.shape

(1796, 2800)

In [136]:
#removed uswrf_sfc_latlon_subset_20080101_20121130

np.savetxt("/home/ec2-user/SageMaker/aws_arch4ML/data/output/Xdata.csv", X, delimiter=",")

In [151]:
precip=nc.Dataset( '/home/ec2-user/SageMaker/aws_arch4ML/data/data/gefs-train/apcp_sfc_latlon_subset_20080101_20121130.nc' )['Total_precipitation'][:]
print(precip.shape)
precip=precip.reshape(1796,11,5,9,16)
precip=np.mean(precip,axis=1)
print(precip.shape)
precip=precip.reshape(1796,9*5*16)
column=['precipcol_'+str(i) for i in range(1,9*5*16+1)]
precip=pd.DataFrame(precip,columns=column)
precip

(1796, 11, 5, 9, 16)
(1796, 5, 9, 16)


,precipcol_1,precipcol_2,precipcol_3,precipcol_4,precipcol_5,precipcol_6,precipcol_7,precipcol_8,precipcol_9,precipcol_10,...,precipcol_711,precipcol_712,precipcol_713,precipcol_714,precipcol_715,precipcol_716,precipcol_717,precipcol_718,precipcol_719,precipcol_720
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015455,0.066364,0.114545,0.103636,0.061818,0.050909
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000909,0.000909,0.001818,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005455,0.045455,0.107273,0.010000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001818,0.024545,0.010000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.030909,0.026364,0.050909,0.045455,0.013636,0.088182,0.180000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025455,0.093636
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.003636,0.063636,0.154545,0.326364,0.165455,...,0.010909,0.002727,0.000000,0.000000,0.009091,0.039091,0.391818,0.524545,0.249091,0.274545
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009091,0.514545,1.600909,...,0.000000,0.000000,0.002727,0.073636,0.380000,0.950909,1.487273,1.416364,1.139091,1.273636
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022727,0.024545,0.003636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.020909,0.330909,1.370909,2.955455,2.987273,1.509091,1.181818


In [ ]:
from numpy import genfromtxt
X = genfromtxt('Xdata.csv', delimiter=',')

In [86]:
#Start by loading one feature to see what we have.  
#Will use pressure as the test file to view and setup the workflow aws_arch4ML/data/data/gefs-train
press_nc = nc.Dataset('/home/ec2-user/SageMaker/aws_arch4ML/data/data/gefs-train/uswrf_sfc_latlon_subset_20080101_20121130.nc') 
print(press_nc)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    title: Subset of data from 2nd-generation multi-decadal ensemble reforecast generated from the NCEP Global Ensemble Forecast System, mimicking version operational at NCEP/EMC circa mid-2012.
    institution: NOAA Earth System Research Laboratory (ESRL)
    source: NCEP GFS v 9.01, T254L42.  Control initial conditions from CFSRR.  Perturbed initial conditions from ETR.  Model error simulated with STTP.
    references: http://www.esrl.noaa.gov/psd/forecasts/reforecast2/index.html
    history: Subset created 2013-01-15 19:20:03 UTC
    comment: Original dataset generated on DOE's supercomputers at Lawrence Berkeley Laboratory through ALCC/ASCR grant.
    dimensions(sizes): time(1796), lat(9), lon(16), ens(11), fhour(5)
    variables(dimensions): float64 time(time), int32 intTime(time), float32 lat(lat), float32 lon(lon), int16 ens(ens), int32 fhour(fhour), int32 intValidTime(t

In [139]:
time = press_nc.variables['time']
lat = press_nc.variables['lat']
lon = press_nc.variables['lon']
ens = press_nc.variables['ens']
fhour = press_nc.variables['fhour']
USWRF = press_nc.variables['Upward_Short-Wave_Rad_Flux']

print(USWRF)

<class 'netCDF4._netCDF4.Variable'>
float32 Upward_Short-Wave_Rad_Flux(time, ens, fhour, lat, lon)
    _FillValue: 9999.0
    units: W m-2
    long_name: Upward_Short-Wave_Rad_Flux_Average (Average for  Mixed Intervals) @ surface
    cell_methods: time: mean
    GRIB_param_discipline: Meteorological_products
    GRIB_param_category: Short-wave_Radiation
    GRIB_param_name: Upward_short_wave_rad_flux
    GRIB_generating_process_type: Forecast
    GRIB_param_id: [  2   0   4 193]
    GRIB_product_definition_template: 8
    GRIB_product_definition_template_desc: Average, accumulation, extreme values or other statistically processed value at a horizontal level in a time interval
    GRIB_level_type: 1
    GRIB_level_type_name: surface
    GRIB_interval_stat_type: Average
    GRIB_VectorComponentFlag: easterlyNortherlyRelative
unlimited dimensions: 
current shape = (1796, 11, 5, 9, 16)
filling on


In [122]:
jd = nc.num2date(time[:], time.units)

ens = pd.DataFrame(Upward_Short-Wave_Rad_Flux[:])

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [ ]:
vname = 'Tx_1211'
station = 0

h = press_nc.variables[vname]
times = press_nc.variables['ens']
jd = nc.num2date(times[:],times.units)
hs = pd.Series(h[:,station],index=jd)

fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(111)
hs.plot(ax=ax,title='%s at %s' % (h.long_name,nc.id))
ax.set_ylabel(h.units)

In [98]:
#This box converts the feature (pressure) into a dictionary. Once I got this working 
#I used this to create a function to pass all features through

#Let's grab that variable and look at the shape
pressure = press_nc.variables.values()

#Create output lists and dictionaries
lat_lon =[]
p_list =[]
press_dict = {}

for l in list(range(len(lat))): #Length of latitude array    
	for n in list(range(len(lon))): #Length of longitude array    
        
		lat_lon.append(np.column_stack((lat[l],lon[n])))

		temp = list(pressure[:,:,:,l,n])
		pcp = np.median(temp, axis=1) #Outputs median of the five models
		#pcp = np.column_stack((t,pcp)) #This adds the time column. Not working.

		#pcp_av = np.median(pcp,axis=1).tolist() #Outputs the median of the median as values
		pcp_av = np.median(pcp,axis=1) #Outputs the median of the median
		#pcp_av = np.column_stack((t,pcp_av)) #This adds the time column. Not working.

        #This converts to csv if required
		#np.savetxt('pcp_'+str(int(lat[l]))+'_'+str(int(lon[n]))+'.csv', pcp, delimiter = ",", fmt = "%.06f" )
		#np.savetxt('pcp_av_'+str(int(lat[l]))+'_'+str(int(lon[n]))+'.csv', pcp_av, delimiter = ",", fmt = "%.06f" )

		p_list.append([[int(lat[l]),int(lon[n])],pcp_av])

#Create data dictionary for precipitation where the key is lat/long and value is the pressure over time
for n in range(len(p_list)):
	k = tuple(p_list[n][0])
	#key = tuple(key)
	v = p_list[n][1]
	press_dict[k] = [v]


TypeError: 'odict_values' object is not subscriptable

In [93]:
#Function that converts netcdf4 to dictionary where key is lat/lon and value is array information in time

def netcdf4_to_dict(data_nc):
    import timeit
    start = timeit.default_timer()
    data = data_nc.variables.values()
    c_list=[]
    c_dict={}

    for l in lat_len:
        for n in lon_len:
            lat_lon.append(np.column_stack((lat[l],lon[n])))

    
            temp = list(data[:,:,:,l,n])
            pcp = np.median(temp, axis=1) #Outputs median of the five models
            #pcp = np.column_stack((t,pcp)) #This adds the time column

            #pcp_av = np.median(pcp,axis=1).tolist() #Outputs the median of the median as values
            pcp_av = np.median(pcp,axis=1) #Outputs the median of the median
            #pcp_av = np.column_stack((t,pcp_av)) #This adds the time column

            c_list.append([[int(lat[l]),int(lon[n])],pcp_av])

    
    # key: is location in latitude and longitude
    #value: is the feature over time incremented in days
    for n in range(len(c_list)):
        k = tuple(c_list[n][0])
        v = c_list[n][1]
        c_dict[k] = [v]
    stop = timeit.default_timer()
    print ("Time: %d s" % (stop - start))
    return c_dict

In [94]:
#Test the function on the Pressure feature.
Press_dict = netcdf4_to_dict(press_nc)

#It worked and took 17s.

NameError: name 'lat_len' is not defined

### 3. Create Train and Test Sets
Now, we'll build 2 datasets. One for training, another for testing. Both need to be written to json files, then copied over to S3.

In [33]:
def get_split(df, freq='D', split_type = 'train', cols_to_use = ['ACME']):
    rt_set = []
    
    # use 70% for training
    if split_type == 'train':
        lower_bound = 0
        upper_bound = round(df.shape[0] * .7)
        
    # use 15% for validation
    elif split_type == 'validation':
        lower_bound = round(df.shape[0] * .7)
        upper_bound = round(df.shape[0] * .85)
        
    # use 15% for test
    elif split_type == 'test':
        lower_bound = round(df.shape[0] * .85)
        upper_bound = df.shape[0]
            
    # loop through columns you want to use
    for h in list(df):
        if h in cols_to_use:
            
            target_column = df[h].values.tolist()[lower_bound:upper_bound]
            
            date_str = str(df.iloc[0]['Date'])
            
            year = date_str[0:4]
            month = date_str[4:6]
            date = date_str[7:]
                                                
            start_dataset = pd.Timestamp("{}-{}-{} 00:00:00".format(year, month, date, freq=freq))
                        
            # create a new json object for each column
            json_obj = {'start': str(start_dataset),
                       'target':target_column}
    
            rt_set.append(json_obj)
    
    return rt_set

In [36]:
train_set = get_split(df)
test_set = get_split(df, split_type = 'test')

In [37]:
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
#             fp.write("\n".encode('utf-8'))

In [38]:
write_dicts_to_file('train.json', train_set)
write_dicts_to_file('test.json', test_set)

In [39]:
!aws s3 cp train.json s3://forecasting-do-not-delete/train/train.json
!aws s3 cp test.json s3://forecasting-do-not-delete/test/test.json

upload: ./train.json to s3://forecasting-do-not-delete/train/train.json
upload: ./test.json to s3://forecasting-do-not-delete/test/test.json


### 4. Run a SageMaker Training Job
Ok! If everything worked, we should be able to train a model in SageMaker straight away.

In [40]:
sess = sagemaker.Session()
region = sess.boto_region_name
image = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
role = sagemaker.get_execution_role()   

In [41]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sess,
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='deepar-electricity-demo',
    output_path='s3://forecasting-do-not-delete/output'
)

In [42]:
hyperparameters = {
    
    # frequency interval is once per day
    "time_freq": 'D',
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    
    # let's use the last 30 days for context
    "context_length": str(30),
    
    # let's forecast for 30 days
    "prediction_length": str(30)
}

estimator.set_hyperparameters(**hyperparameters)

In [ ]:
data_channels = {
    "train": "s3://forecasting-do-not-delete/train/train.json",
    "test": "s3://forecasting-do-not-delete/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

### 5. Run Inference
If you made it this far, congratulations! None of this is easy. For your next steps, please open up the example notebook under the SageMakerExamples:
- SageMakerExamples/Introduction To Amazon Algorithms/DeepAR-Electricity.

That will walk you through both how to add more timeseries to your model, and how to get inference results out of it.

### 6. Extend Your Solution
Now you're getting forecasts, how will you extend your solution? How good are your forecasts? What about getting forecasts for the other stations? Is your model cognizant of the weather?

Spend your remaining time growing your modeling solution to leverage additional datasets. Then, think through how you'd set this up to run in production.